### 417. Pacific Atlantic Water Flow

**時間複雜度: $O( m * n )$**  
**空間複雜度: $O( m * n )$**

#### DFS

In [1]:
from typing import List

class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        # 取得矩陣的行數與列數
        rows = len(heights)
        cols = len(heights[0])
        
        # 儲存最終結果的座標（可同時流向太平洋與大西洋的格子）
        result = []
        
        # 分別記錄可流向太平洋與大西洋的格子集合
        pacific = set() # space: O(m x n)
        atlantic = set() # space: O(m x n)
        
        # 四個方向（上、下、左、右）
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        # 深度優先搜尋（DFS）函式
        def dfs(row, col, visited, previous_height):
            # 若該格超出邊界、已訪問過、或高度比前一格低則停止搜尋
            if (
                (row, col) in visited or
                row < 0 or col < 0 or
                row == rows or col == cols or
                heights[row][col] < previous_height
            ):
                return
            
            # 標記該格為已訪問
            visited.add((row, col))
            
            # 向四個方向繼續搜尋
            for drow, dcol in directions:
                dfs(row + drow, col + dcol, visited, heights[row][col])
            

        # 以與海洋相鄰的邊界作為起點開始 DFS
        # 每個格子最多被太平洋和大西洋各訪問一次，time: O(2 × (m × n)) = O(m x n)

        for row in range(rows):
            # 與太平洋相連的左側邊界（第一欄）
            dfs(row, 0, pacific, heights[row][0])

            # 與大西洋相連的右側邊界（最後一欄）
            dfs(row, cols-1, atlantic, heights[row][cols-1])

        for col in range(cols):
            # 與太平洋相連的上側邊界（第一行）
            dfs(0, col, pacific, heights[0][col])

            # 與大西洋相連的下側邊界（最後一行）
            dfs(rows-1, col, atlantic, heights[rows-1][col])

        # 檢查每個格子，若同時可流向兩個海洋則加入結果
        for row in range(rows):
            for col in range(cols):
                if (row, col) in pacific and (row, col) in atlantic:
                    result.append([row, col])

        # 回傳所有符合條件的座標
        return result


In [2]:
heights = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
# Output: [[0,4],[1,3],[1,4],[2,2],[3,0],[3,1],[4,0]]
Solution().pacificAtlantic(heights)

[[0, 4], [1, 3], [1, 4], [2, 2], [3, 0], [3, 1], [4, 0]]

#### BFS

In [3]:
from typing import List
from collections import deque

class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        # 取得矩陣的行數與列數
        rows = len(heights)
        cols = len(heights[0])

        # 定義四個方向（上、下、左、右）
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        # 分別建立兩個布林矩陣，紀錄太平洋與大西洋可到達的格子
        pacific = [[False] * cols for _ in range(rows)]
        atlantic = [[False] * cols for _ in range(rows)]

        # 定義 BFS 函式，用來從邊界向內擴散
        def bfs(source, ocean):
            # 使用雙端佇列來進行 BFS
            queue = deque(source)

            while queue:
                # 取出目前的座標
                row, col = queue.popleft()
                # 標記此格子可到達該海域
                ocean[row][col] = True

                # 檢查四個方向的鄰居格子
                for dr, dc in directions:
                    new_row = row + dr
                    new_col = col + dc

                    # 確保新座標在矩陣範圍內，且尚未訪問過
                    # 並且高度要「大於等於」目前的格子，水才能逆流回來
                    if (
                        0 <= new_row < rows and
                        0 <= new_col < cols and
                        not ocean[new_row][new_col] and
                        heights[new_row][new_col] >= heights[row][col]
                    ):
                        # 若符合條件，將新座標加入佇列
                        queue.append((new_row, new_col))

        # 建立太平洋與大西洋的邊界起始點
        pacific_source = []
        atlantic_source = []

        # 太平洋：左邊界與上邊界
        # 大西洋：右邊界與下邊界
        for row in range(rows):
            pacific_source.append((row, 0))        # 太平洋左側
            atlantic_source.append((row, cols-1))  # 大西洋右側

        for col in range(cols):
            pacific_source.append((0, col))        # 太平洋上側
            atlantic_source.append((rows-1, col))  # 大西洋下側

        # 分別從兩個海洋邊界進行 BFS 擴散
        bfs(pacific_source, pacific)
        bfs(atlantic_source, atlantic)

        # 收集同時能流向太平洋與大西洋的格子座標
        result = []
        for row in range(rows):
            for col in range(cols):
                if pacific[row][col] and atlantic[row][col]:
                    result.append((row, col))

        return result

In [4]:
heights = [[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]]
# Output: [[0,4],[1,3],[1,4],[2,2],[3,0],[3,1],[4,0]]
Solution().pacificAtlantic(heights)

[(0, 4), (1, 3), (1, 4), (2, 2), (3, 0), (3, 1), (4, 0)]